In [ ]:
import optuna
from ultralytics import YOLO

study = optuna.create_study(direction="maximize")

def trial_logic(trial):
    model_next_iter = YOLO("../../yolo11n.pt")  # or yolov11 if you have it locally

    result = model_next_iter.train(
        data="traffic_data/traffic_data/data_1.yaml",
        epochs=50,
        imgsz=640,
        batch=16,
        optimizer="auto",
        lr0=trial.suggest_loguniform("lr0", 1e-5, 1e-2),
        patience=10,
        amp=True,
        hsv_h=trial.suggest_float("hsv_h", 0.0, 0.05),
        hsv_s=trial.suggest_float("hsv_s", 0.5, 0.9),
        hsv_v=trial.suggest_float("hsv_v", 0.3, 0.7),
        degrees=5.0,
        translate=0.1,
        scale=trial.suggest_float("scale", 0.3, 0.6),
        fliplr=0.5,
        mosaic=1.0,
        project="traffic_training",
        name="traffic_yolo11n_optuna",
        save=True,
        plots=True
    )

    return result.metrics["metrics/mAP50"]

study.optimize(trial_logic, n_trials=20)

print("Best hyperparameters:", study.best_params)
print("Best mAP50-95 score:", study.best_value)


In [ ]:
def trial_logic(trial):
    model = YOLO("../../yolo11n.pt")

    model.train(
        data="traffic_data/traffic_data/data_1.yaml",
        epochs=50,
        imgsz=640,
        batch=16,
        optimizer="auto",
        lr0=trial.suggest_loguniform("lr0", 1e-5, 1e-2),
        patience=10,
        amp=True,
        hsv_h=trial.suggest_float("hsv_h", 0.0, 0.05),
        hsv_s=trial.suggest_float("hsv_s", 0.5, 0.9),
        hsv_v=trial.suggest_float("hsv_v", 0.3, 0.7),
        scale=trial.suggest_float("scale", 0.3, 0.6),
        fliplr=0.5,
        mosaic=1.0,
        project="traffic_training",
        name="traffic_yolo11n_optuna2",
        save=True,
        plots=False
    )

    # Access trainer metrics directly
    try:
        return model.trainer.results_dict["metrics/mAP50-95"]
    except Exception:
        raise optuna.TrialPruned()  # if metric not found, prune the trial


In [ ]:
from ultralytics import YOLO

# Load model
model = YOLO("../../yolo11n.pt")

# Launch hyperparameter evolution
model.tune(
    data="traffic_data/data_1.yaml",  # your dataset
    epochs=40,
    iterations=10,
    device=0,# how many combinations to try
    imgsz=640,
    batch=16,
    optimizer="auto",               # or 'SGD', 'Adam', 'AdamW'
    project="traffic_training",
    name="yolo11n_tune",
    cache=True                      # cache images for speed
)
